In [2]:
import cv2
import numpy as np
import os
import csv 

from utils import *

In [ ]:
"Bước 1: Lấy mẫu video, frame_distance là tham số khoảng cách giữa 2 lần lấy mẫu liên tiếp"

video_path_1 = "DJI_0044.mp4"
video_path_2 = "DJI_0046.mp4"
frame_distance = 24
output_folder_1 = "v44"
output_folder_2 = "v46"
extract_frames(video_path_1, frame_distance, output_folder_1, resize=0.5)
extract_frames(video_path_2, frame_distance, output_folder_2, resize=0.5)

In [6]:
"""
Bước 2: Từ các khung hình đã lấy mẫu từ trước, tính toán ma trận H giữa 2 khung liên tiếp
Sử dụng thuật toán akaze dùng hàm "findH_akaze"
Sử dụng thuật toán sift dùng hàm "findH_sift"
Sử dụng thuật toán orb dùng hàm "findH_orb"
"""

folder_path_A = "v44"
image_files_A = os.listdir(folder_path_A)
image_files_A.sort()
image_A = [cv2.imread(os.path.join(folder_path_A,img)) for img in image_files_A]

folder_path_B = "v46"
image_files_B = os.listdir(folder_path_B)
image_files_B.sort()
image_B = [cv2.imread(os.path.join(folder_path_B,img)) for img in image_files_B]

image_mix = image_A + image_B
image_0 = image_mix [0]
H = []

for i, image in enumerate(image_mix[1:], 1):
    next_image = image
    homo = findH_akaze(image_0, image)
    image_0 = next_image
    H.append(homo)

csv_file = "homographies_mix_akaze.csv"
with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    for i, homo in enumerate(H):
        writer.writerow([f"H_{i}"])
        for row in homo:
            writer.writerow(row)
    print(f"Homographies have been saved to {csv_file}")

Homographies have been saved to homographies_mix_akaze.csv


In [7]:
"""
Đọc ma trận H từ file đã lưu
Tính toán kích thước ảnh đầu ra
Bắt đầu ghép ảnh và xuất file
"""
output_width, output_height, min_x, min_y = calculate_output_size(image_mix, H)
output_shape = (output_width, output_height)
print(output_shape,min_x,min_y)

result = np.zeros((output_shape[1], output_shape[0], 3), dtype=np.uint8)

for i, image in enumerate(image_mix):
    warped_image = warp_image_to_zero(image, H, i, output_shape, min_x, min_y)
    result = np.maximum(result, warped_image)
    cv2.imwrite("mix.jpg", result)

(6372, 6491) -4936.995 -4520.843
